#  Hands-on-session1 (Querying data from materials databases)**

# **Necessary Libraries**

In [1]:
!pip install matminer[citrine]
!pip install pyyaml
!pip install mp_api
!pip install pandas==2.2.2
!git clone https://github.com/RealBack1/practice.git

fatal: destination path 'practice' already exists and is not an empty directory.


# **Practice**

**Getting Data**

*Revised from Joseph Montoya's notebook*

This notebook demonstrates a few basic examples from matminer's data retrieval features. Matminer supports data retrieval from the following sources.

*   Materials Project (https://materialsproject.org/)

This notebook was last updated 11/15/18 for version 0.4.5 of matminer.

Each resource has a corresponding object in matminer designed for retrieving data and preprocessing it into a pandas dataframe. In addition, matminer can also access and aggregate data from your own mongo database, if you have one.


**Data retrieval**

The materials project data retrieval tool, matminer.data_retrieval.retrieve_MP.MPDataRetrieval is initialized using an api_key that can be found on your personal dashboard page on materialsproject.org if you've created an account. If you've set your api key via pymatgen (e.g. pmg config --add PMG_MAPI_KEY YOUR_API_KEY_HERE), the data retrieval tool may be initialized without an input argument.

**We need our own MP API Key**

In [2]:
## Materials Project API client: https://docs.materialsproject.org/downloading-data/using-the-api/getting-started
## Loading module of MP API client
from mp_api.client import MPRester
import pandas as pd

/home/seungjobaek/miniconda3/envs/Class_Practice/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## Put your own API key
my_api_key = ""
mpr = MPRester(my_api_key)

In [5]:
list_of_available_fields = mpr.materials.summary.available_fields
print(list_of_available_fields)

['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal_anisotropy', 'homogeneous_poisson', 'e_total', 'e_ionic', 'e_electronic',

**Practice 1: Get various properties of binary oxide materials with "AO" formula**

In [ ]:
docs = mpr.materials.summary.search(chemsys="*-O", formula="AB", fields=list_of_available_fields)
results = [doc.dict() for doc in docs]
df_full = pd.DataFrame(results)
# df_full = pd.read_excel("/content/practice/Class/data/Hands_on_session1_data.xlsx", sheet_name="example_1")
df = df_full.loc[:, ["formula_pretty", "material_id", "formation_energy_per_atom", "energy_above_hull", "band_gap"]]
sdf = df.sort_values(by="formation_energy_per_atom", axis=0)
sdf.head(10)

**Practice 2: Draw the phase diagram for Li-Ni-O**

(This code was written with reference to the Jupyter notebook by Materials Virtual Lab(https://matgenb.materialsvirtuallab.org/))

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
entries = mpr.get_entries_in_chemsys(elements=["Li", "Ni", "O"], additional_criteria={"thermo_types": ["GGA_GGA+U"]})
pd = PhaseDiagram(entries)
plotter = PDPlotter(pd, backend="matplotlib")
plotter.show()

**Practice 3: Among the Li–X–O compounds, identify those with the R-3m crystal structure, and classify the stable ones (with energy_above_hull = 0)**

In [7]:
pd.set_option("display.max_rows", None)   # None → 제한 없음
pd.set_option("display.max_columns", None)

docs = mpr.materials.summary.search(chemsys="Li-*-O", num_elements=3, spacegroup_symbol="R-3m", fields=list_of_available_fields)
results = [doc.dict() for doc in docs]
df_full = pd.DataFrame(results)
# df_full = pd.read_excel("/content/practice/Class/data/Hands_on_session1_data.xlsx", sheet_name="example_4")
df = df_full.loc[:, ["formula_pretty", "material_id", "energy_above_hull","band_gap", "symmetry"]]
df.head()

MPRestError: REST query returned with error status code 401 on URL https://api.materialsproject.org/materials/summary/?nelements_min=3&nelements_max=3&spacegroup_symbol=R-3m&_fields=builder_meta%2Cnsites%2Celements%2Cnelements%2Ccomposition%2Ccomposition_reduced%2Cformula_pretty%2Cformula_anonymous%2Cchemsys%2Cvolume%2Cdensity%2Cdensity_atomic%2Csymmetry%2Cproperty_name%2Cmaterial_id%2Cdeprecated%2Cdeprecation_reasons%2Clast_updated%2Corigins%2Cwarnings%2Cstructure%2Ctask_ids%2Cuncorrected_energy_per_atom%2Cenergy_per_atom%2Cformation_energy_per_atom%2Cenergy_above_hull%2Cis_stable%2Cequilibrium_reaction_energy_per_atom%2Cdecomposes_to%2Cxas%2Cgrain_boundaries%2Cband_gap%2Ccbm%2Cvbm%2Cefermi%2Cis_gap_direct%2Cis_metal%2Ces_source_calc_id%2Cbandstructure%2Cdos%2Cdos_energy_up%2Cdos_energy_down%2Cis_magnetic%2Cordering%2Ctotal_magnetization%2Ctotal_magnetization_normalized_vol%2Ctotal_magnetization_normalized_formula_units%2Cnum_magnetic_sites%2Cnum_unique_magnetic_sites%2Ctypes_of_magne

In [ ]:
df

In [ ]:
df_zero = df[df["energy_above_hull"] == 0]
df_zero